In [36]:
# Ý tưởng:
# Biểu diễn mỗi ký tự bằng one-hot
# Ở mỗi bước t:
#     Thực hiện tính công thức trạng thái ẩn
#     Đầu ra đưa qua softmax
#     loss là cross-entropy loss
# Backpropagation: Tính gradient từ T -> 1, cộng dồn qua thời gian
# Có gradient clipping tránh nổ gradient
import numpy as cp

In [37]:
def one_hot(idx, vocab_size):
    # vector v có vocab_size hàng và 1 cột
    v = cp.zeros((vocab_size, 1))
    # Cả hàng idx sẽ toàn giá trị 1.0
    v[idx] = 1.0
    return v
def softmax(z):
    exp = cp.exp(z)
    return exp / cp.sum(exp)


In [38]:
v = cp.zeros((3, 2))
v[1] = 1.0
v

array([[0., 0.],
       [1., 1.],
       [0., 0.]])

In [39]:
# RNN thuần
class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # Khởi tạo trọng số gồm value + và -
        s = 0.01
        # ma trận gồm hidden_size hàng, input_size cột
        self.Wxh = cp.random.randn(hidden_size, input_size) * s
        # ma trận gồm hidden_size hàng, hidden_size cột
        self.Whh = cp.random.randn(hidden_size, hidden_size) * s
        self.bh = cp.zeros((hidden_size, 1))
        # ma trận gồm output_size hàng, hidden_size cột
        self.Why = cp.random.randn(output_size, hidden_size) * s
        self.by = cp.zeros((output_size, 1))

        #Adam
        self.m = {k: cp.zeros_like(v) for k, v in self.params().items()}
        self.v = {k: cp.zeros_like(v) for k, v in self.params().items()}
        self.t = 0

    def params(self):
        return {
            "Wxh": self.Wxh, "Whh": self.Whh, "bh": self.bh,
            "Why": self.Why, "by": self.by
        }
    def forward(self, inputs, prev_hidden_state):
        # inputs: list chỉ số của ký tự [x1, x2,...]
        # prev_hidden_state : hidden state ban đầu
        # return cache(xs, hs, os, ps) và hidden last
        # xs, hs, os, ps chứa các xt, ht, ot, pt trong công thức của RNN
        xs, hs, os, ps = {}, {}, {}, {}
        # hidden state trước t = 0
        hs[-1] = cp.copy(prev_hidden_state)
        loss = 0.0
        for t, ix in enumerate(inputs):
            xs[t] = one_hot(ix, self.input_size)
            hs[t] = cp.tanh(self.Wxh @ xs[t] + self.Whh @ hs[t-1] + self.bh)
            os[t] = self.Why @ hs[t] + self.by
            ps[t] = softmax(os[t])
        cache = (xs, hs, os, ps)
        # trả về hidden state cuối cùng làm input cho batch tiếp theo
        # trả về cache để tính backpropagation
        return cache, hs[len(inputs)-1]
    def loss_and_grads(self, inputs, targets, prev_hidden_state):
        (xs, hs, os, ps), hlast = self.forward(inputs, prev_hidden_state=prev_hidden_state)
        loss = 0.0
        # Tính loss
        for t in range(len(inputs)):
            loss += -cp.log(ps[t][targets[t],0] + 1e-12)
        # Khởi tạo grads, tạo ra ma trận 0 giống với các ma trận trong ngoặc
        dWxh = cp.zeros_like(self.Wxh)
        dWhh = cp.zeros_like(self.Whh)
        dbh = cp.zeros_like(self.bh)
        dWhy = cp.zeros_like(self.Why)
        dby = cp.zeros_like(self.by)
        dh_next = cp.zeros_like(hs[0])

        # Backpropagation through time
        for t in reversed(range(len(inputs))):
            dy = cp.copy(ps[t])
            dy[targets[t]] -= 1.0
            dWhy += dy @ hs[t].T
            dby += dy

            dh = self.Why.T @ dy + dh_next
            dh_raw = (1 - hs[t] * hs[t]) * dh      # tanh'
            dbh  += dh_raw
            dWxh += dh_raw @ xs[t].T
            dWhh += dh_raw @ hs[t-1].T
            dh_next = self.Whh.T @ dh_raw
        
        # clipping tránh nổ gradient
        for dparam in [dWxh, dWhh, dWhy, dby]:
            cp.clip(dparam, -5, 5, out=dparam)
        
        grads = {"Wxh":dWxh, "Whh":dWhh, "bh":dbh, "Why": dWhy, "by": dby}
        # loss tổng lỗi của chuỗi
        # grads dictionary chứa các gradient để cập nhật trọng số
        # hlast là hidden cuối cùng làm prev hidden state cho batch tiếp theo
        return loss, grads, hlast
    
    # Cập nhật trọng số dùng Adam
    def step(self, grads, lr=1e-2, beta1=0.9, beta2=0.999, eps=1e-8):
        # Số lần update (để hiệu chỉnh bias correction)
        self.t += 1
        # Cập nhật trọng số
        for k in self.params().keys():
            g = grads[k]
            self.m[k] = beta1 * self.m[k] + (1-beta1) * g
            self.v[k] = beta2 * self.v[k] + (1-beta2) * (g * g)

            m_hat = self.m[k] / (1 - beta1**self.t)
            v_hat = self.v[k] / (1 - beta2**self.t)

            self.params()[k] -= lr * m_hat / (cp.sqrt(v_hat) + eps)
    # Sinh chuỗi có độ dài n từ 1 kí tự bất kì nhập vào   
    def sample(self, seed_ix, n, h= None):
        if h is None:
            h = cp.zeros((self.hidden_size, 1))
        x = one_hot(seed_ix, self.input_size)
        # Kết quả cuối cùng chứa các index của các kí tự được chọn (dự đoán)
        indexes = [seed_ix]
        for _ in range(n):
            h = cp.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            o = self.Why @ h + self.by
            p = softmax(o)
            # Bốc thăm kí tự dựa trên phân phối xác suất
            ix = cp.random.choice(range(self.output_size), p= p.ravel())
            # Biểu diễn kí tự được dự đoán thành one-hot để mô hình dự đoán tiếp
            x = one_hot(ix, self.input_size)
            indexes.append(ix)
        return indexes

In [40]:
wxh = cp.random.randn(3, 5)
wxh

array([[-1.21994411, -0.32104962,  0.66782469, -0.55629341, -1.82661296],
       [ 0.60328169, -0.15132807, -0.53859219, -1.02658693,  0.8673077 ],
       [-0.33601004, -0.87111639, -0.92508016,  0.26439911, -0.20874952]])

In [41]:
Wxh = cp.random.randn(3, 5)
print(Wxh)
dwxh = cp.zeros_like(Wxh)
print("---------")
print(dwxh)

[[-0.17642223  1.94636835  0.76602002  0.8546308   0.24590151]
 [ 0.52712006 -0.03315179 -0.69774074  0.84573914  0.77617283]
 [ 0.01129675  0.79674813  0.2155056   1.13595077 -1.06786047]]
---------
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [42]:
s = 0.01
input_size = 4
hidden_size = 3
output_size = input_size
# ma trận gồm hidden_size hàng, input_size cột
Wxh = cp.random.randn(hidden_size, input_size) * s
print("Wxh (hidden size, input size): \n",Wxh,"\n")
# ma trận gồm hidden_size hàng, hidden_size cột
Whh = cp.random.randn(hidden_size, hidden_size) * s
print("Whh (hidden size, hidden size):\n",Whh, "\n")
bh = cp.zeros((hidden_size, 1))
print("bh (hidden size):\n",bh, "\n")
# ma trận gồm output_size hàng, hidden_size cột
Why = cp.random.randn(output_size, hidden_size) * s
print("Why (output size, hidden size): \n",Why, "\n")
by = cp.zeros((output_size, 1))
print("by (output size): \n",by, "\n")

Wxh (hidden size, input size): 
 [[-0.00543693  0.00447066 -0.00202294 -0.0135067 ]
 [ 0.00265306 -0.02379214 -0.00586958  0.01087894]
 [-0.00903648  0.00258772 -0.00062291  0.01692163]] 

Whh (hidden size, hidden size):
 [[-0.01504993 -0.00187612 -0.00179002]
 [-0.00707434 -0.019371   -0.00484082]
 [ 0.00459155 -0.010987    0.00503149]] 

bh (hidden size):
 [[0.]
 [0.]
 [0.]] 

Why (output size, hidden size): 
 [[-6.11869220e-03  2.03087198e-03 -1.11105845e-02]
 [ 8.93357826e-05 -1.74451905e-02  2.78555305e-03]
 [-1.25487413e-02  1.28556759e-02 -6.80950409e-03]
 [ 7.03537738e-04 -7.41783011e-03  4.92635906e-03]] 

by (output size): 
 [[0.]
 [0.]
 [0.]
 [0.]] 



In [43]:
inputs = [1, 2, 3]
input_size = 4
xs, hs, os, ps = {}, {}, {}, {}
hs[-1] = cp.zeros((hidden_size,1))
for t, ix in enumerate(inputs):
    xs[t] = one_hot(ix, input_size)
    hs[t] = cp.tanh(Wxh @ xs[t] + Whh @ hs[t-1] + bh)
    os[t] = Why @ hs[t] + by
    ps[t] = softmax(os[t])
    print(f"Giá trị thứ {t} của input là:\n{xs[t]}\nCó hidden state thứ {t} là:\n{hs[t]}")
    print(f"Output thứ {t} của input thứ {t} là:\n {os[t]}\nCó xác suất:\n {ps[t]}")
    print("---------------------------------------------------------")

Giá trị thứ 0 của input là:
[[0.]
 [1.]
 [0.]
 [0.]]
Có hidden state thứ 0 là:
[[ 0.00447063]
 [-0.02378765]
 [ 0.00258771]]
Output thứ 0 của input thứ 0 là:
 [[-0.00010442]
 [ 0.00042259]
 [-0.00037953]
 [ 0.00019235]]
Có xác suất:
 [[0.2499657 ]
 [0.25009747]
 [0.24989694]
 [0.25003989]]
---------------------------------------------------------
Giá trị thứ 1 của input là:
[[0.]
 [0.]
 [1.]
 [0.]]
Có hidden state thứ 1 là:
[[-0.00205022]
 [-0.00545289]
 [-0.00032801]]
Output thứ 1 của input thứ 1 là:
 [[ 5.11493559e-06]
 [ 9.40298951e-05]
 [-4.21393457e-05]
 [ 3.73903202e-05]]
Có xác suất:
 [[0.24999538]
 [0.25001761]
 [0.24998357]
 [0.25000345]]
---------------------------------------------------------
Giá trị thứ 2 của input là:
[[0.]
 [0.]
 [0.]
 [1.]]
Có hidden state thứ 2 là:
[[-0.01346422]
 [ 0.01100022]
 [ 0.01696885]]
Output thứ 2 của input thứ 2 là:
 [[-8.38104309e-05]
 [-1.45836121e-04]
 [ 1.94824744e-04]
 [-7.47568659e-06]]
Có xác suất:
 [[0.24998169]
 [0.24996618]
 [0.2500

In [44]:
# Tạo các giá trị về dữ liệu huấn luyện
text = "Nice to meet you. I am from Vietnam. Vietnam is a beautiful country. You should come"
# lặp lại 100 lần cho đủ dài
text = (text * 100)

# Sắp xếp tăng dần các kí tự unique trong text
chars = sorted(list(set(text)))
# Tập từ vựng có số lượng bằng len(chars)
vocab_size = len(chars)
# Tạo từ điển char -> index
ch2i = {ch:i for i, ch in enumerate(chars)}
# Tạo từ điển index -> char
i2ch = {i:ch for ch, i in ch2i.items()}

# Biến chuỗi text thành các index
data = [ch2i[c] for c in text]

# Hàm chuyển chỉ số sang kí tự
def to_string(indexes):
    return "".join(i2ch[i] for i in indexes)


In [45]:
# Mô hình 
hidden_size = 32
model = RNN(vocab_size, hidden_size=hidden_size, output_size=vocab_size)
# Mỗi lần model đọc 1 đoạn con dài 25 kí tự (mini batch)
seq_len = 25
lr = 5e-2
# p là con trỏ đọc dữ liệu
p = 0
smooth_loss = None
prev_hidden_state = cp.zeros((hidden_size, 1))

epochs = 2000
for n in range(epochs):
    # Nếu đã đọc hết dữ liệu thì reset lại prev_hidden_state và p
    if p + seq_len + 1 >= len(data):
        p = 0
        prev_hidden_state = cp.zeros((hidden_size, 1))
    # minibatch 1 chuỗi liên tục
    inputs = data[p: (p+seq_len)]
    targets = data[p+1:(p+seq_len+1)]

    # Tính loss, gradient, trạng thái cuối của batch đầu làm trạng thái đầu cho batch sau
    loss, grads, prev_hidden_state = model.loss_and_grads(inputs, targets, prev_hidden_state)
    # Cập nhật trọng số
    model.step(grads=grads, lr=lr)
    # Làm loss mượt hơn (giữ 99% giá trị cũ, thêm 1% giá trị mới => ổn định, tránh dao động mạnh)
    if smooth_loss is None:
        smooth_loss = loss
    else:
        smooth_loss = 0.99 * smooth_loss + 0.01*loss
    # Sinh thử văn bản
    if n % 200 == 0:
        sample_index = model.sample(seed_ix=inputs[0], n = 60)
        print(f"bước {n:4d} | loss = {smooth_loss:.3f} | sample: {to_string(sample_index)}")
    p += seq_len
    


bước    0 | loss = 78.386 | sample: Nyei YyVtNeemrdd Nuou. YrbIilontsfb.yadcYYsmthbsumlnsrtVoftes
bước  200 | loss = 12.839 | sample:  comeNice to meet you. I am from Vietnam. Vietnam is a beauti
bước  400 | loss = 1.744 | sample: ietnam. Vietnam is a beautiful country. You should comeNice t
bước  600 | loss = 0.245 | sample: s am is a beautiful country. You should comeNice to meet you.
bước  800 | loss = 0.040 | sample: l country. You should comeNice to meet you. I am from Vietnam
bước 1000 | loss = 0.010 | sample: I am from Vietnam. Vietnam is a beautiful country. You should
bước 1200 | loss = 0.005 | sample: e uoomeNice to meet you. I am from Vietnam. Vietnam is a beau
bước 1400 | loss = 0.003 | sample:  cou. I am from Vietnam. Vietnam is a beautiful country. You 
bước 1600 | loss = 0.002 | sample: ntnam is a beautiful country. You should comeNice to meet you
bước 1800 | loss = 0.002 | sample:  cou. I am from Vietnam. Vietnam is a beautiful country. You 
